[View in Colaboratory](https://colab.research.google.com/github/misttc/MockObject/blob/master/Copy_of_Copy_of_S2S_E2F.ipynb)

# Setup local drive

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1XDU6OEibG-koafLubiYdWkTncBkxyEG4'
downloaded = drive.CreateFile({'id': file_id})
downloaded = drive.CreateFile({'id':'1XDU6OEibG-koafLubiYdWkTncBkxyEG4'}) # replace the id with id of file you want to access
downloaded.GetContentFile('fra.txt')  

# Read file as panda dataframe
# import pandas as pd
# xyz = pd.read_csv('xyz.csv')

# Setup Keras

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

Using TensorFlow backend.


# Vectorize the data.

In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Show the data

In [0]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.

In [0]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn

In [0]:
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 28s 4ms/step - loss: 0.9274 - val_loss: 0.9880
Epoch 2/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.7433 - val_loss: 0.8023
Epoch 3/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.6262 - val_loss: 0.7155
Epoch 4/100
1600/8000 [=====>........................] - ETA: 19s - loss: 0.5821

8000/8000 [==============================] - 26s 3ms/step - loss: 0.5682 - val_loss: 0.6798
Epoch 5/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.5287 - val_loss: 0.6250
Epoch 6/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.4956 - val_loss: 0.5930
Epoch 7/100
6144/8000 [======================>.......] - ETA: 5s - loss: 0.4690

8000/8000 [==============================] - 26s 3ms/step - loss: 0.4692 - val_loss: 0.5697
Epoch 8/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.4472 - val_loss: 0.5552
Epoch 9/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.4283 - val_loss: 0.5400
Epoch 10/100
6848/8000 [========================>.....] - ETA: 3s - loss: 0.4109

8000/8000 [==============================] - 26s 3ms/step - loss: 0.4108 - val_loss: 0.5252
Epoch 11/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.3947 - val_loss: 0.5151
Epoch 12/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.3798 - val_loss: 0.5080
Epoch 13/100
6784/8000 [========================>.....] - ETA: 3s - loss: 0.3659

8000/8000 [==============================] - 26s 3ms/step - loss: 0.3659 - val_loss: 0.5006
Epoch 14/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.3529 - val_loss: 0.4944
Epoch 15/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.3405 - val_loss: 0.4859
Epoch 16/100
6848/8000 [========================>.....] - ETA: 3s - loss: 0.3293

...

8000/8000 [==============================] - 26s 3ms/step - loss: 0.0642 - val_loss: 0.7441
Epoch 92/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0637 - val_loss: 0.7489
Epoch 93/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0626 - val_loss: 0.7448
Epoch 94/100
6912/8000 [========================>.....] - ETA: 3s - loss: 0.0612

8000/8000 [==============================] - 26s 3ms/step - loss: 0.0619 - val_loss: 0.7521
Epoch 95/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0611 - val_loss: 0.7528
Epoch 96/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0601 - val_loss: 0.7662
Epoch 97/100
6784/8000 [========================>.....] - ETA: 3s - loss: 0.0592

8000/8000 [==============================] - 26s 3ms/step - loss: 0.0596 - val_loss: 0.7641
Epoch 98/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0587 - val_loss: 0.7625
Epoch 99/100
8000/8000 [==============================] - 27s 3ms/step - loss: 0.0576 - val_loss: 0.7700
Epoch 100/100
6784/8000 [========================>.....] - ETA: 3s - loss: 0.0566

8000/8000 [==============================] - 26s 3ms/step - loss: 0.0572 - val_loss: 0.7671


/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# decode sequence

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Output

In [0]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Suite-le.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: I see.
Decoded sentence: Je comprends.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: Oh